In [13]:
from gigachat import GigaChat
from getpass import getpass
import numpy as np
from nltk.util import ngrams

In [2]:
creds = getpass()

In [5]:
model = GigaChat(credentials=creds, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)

In [65]:
test_sent = "его оригинальный портрет выставлен на продажу".split()

test_phrase = "подлинная картина"

In [58]:
bigram = ngrams(test_sent, 2)
trigram = ngrams(test_sent, 3)
test_sent_bigram = [" ".join(words) for words in bigram]
test_sent_trigram = [" ".join(words) for words in trigram]

In [59]:
sent_embeds = model.embeddings(test_sent).data
phrase_embed = model.embeddings(test_phrase).data[0].embedding
sims = []
for word in sent_embeds:
    sims.append(np.dot(word.embedding / np.linalg.norm(word.embedding), phrase_embed / np.linalg.norm(phrase_embed)))
sims

[np.float64(0.846681095569723),
 np.float64(0.8387825615295545),
 np.float64(0.8617398385012418),
 np.float64(0.824595308464789),
 np.float64(0.9381885005310974)]

In [60]:
sent_embeds = model.embeddings(test_sent_bigram).data
phrase_embed = model.embeddings(test_phrase).data[0].embedding
sims = []
for word in sent_embeds:
    sims.append(np.dot(word.embedding / np.linalg.norm(word.embedding), phrase_embed / np.linalg.norm(phrase_embed)))
sims

[np.float64(0.8415364096300624),
 np.float64(0.8212341917342143),
 np.float64(0.7934345392815161),
 np.float64(0.8539167719926526)]

In [61]:
sent_embeds = model.embeddings(test_sent_trigram).data
phrase_embed = model.embeddings(test_phrase).data[0].embedding
sims = []
for word in sent_embeds:
    sims.append(np.dot(word.embedding / np.linalg.norm(word.embedding), phrase_embed / np.linalg.norm(phrase_embed)))
sims

[np.float64(0.8092990195986802),
 np.float64(0.7627808059596929),
 np.float64(0.7990036833298929)]

In [66]:
sent_embeds = model.embeddings(" ".join(test_sent)).data
phrase_embed = model.embeddings(test_phrase).data[0].embedding
sims = []
for word in sent_embeds:
    sims.append(np.dot(word.embedding / np.linalg.norm(word.embedding), phrase_embed / np.linalg.norm(phrase_embed)))
sims

[np.float64(0.8464416161209178)]

# Tests

In [2]:
import pandas as pd

In [32]:
example_df = pd.read_csv("data/example.csv", sep=";")
example_df.head()

,id,sentence,phrase,target
0,1,доченька твоя совсем большая стала,дочь,доченька
1,2,вся дорога забита деревьями и цветами,дерево,деревьями
2,3,в следующее воскресенье я собираюсь в питер,санкт петербург,питер
3,4,у меня сломалась стиралка прикинь,стиральная машина,стиралка
4,5,садись в машину и поехали уже,автомобиль,машину


In [38]:
test_df = pd.read_csv("data/synth_set.csv", sep=";")
test_df

,id,sentence,phrase,target
0,0,доченька твоя совсем большая стала,дочь,доченька
1,1,вся дорога забита деревьями и цветами,дерево,деревьями
2,2,в следующее воскресенье я собираюсь в питер,санкт петербург,питер
3,3,у меня сломалась стиралка прикинь,стиральная машина,стиралка
4,4,ты возьми корзину прежде чем набрать продукты,звонить,NaN
...,...,...,...,...
995,995,на столе стоял кувшин с водичкой для питья,вода,водичкой
996,996,малыш радостно смеялся новой погремушке,младенец,малыш
997,997,этот регион славится своими винами и кухней,область,регион
998,998,он починил свой старый мобильник в мастерской,сотовый телефон,мобильник


In [33]:
from sem_sim.use_cases.sent_processor import SentProcessor
from tqdm import tqdm
import numpy as np
import math

In [5]:
processor = SentProcessor()

In [30]:
sents = example_df["sentence"].to_list()
phrases = example_df["phrase"].to_list()
targets = example_df["target"].to_list()

In [8]:
preds = []
for sent, phrase in tqdm(zip(sents, phrases), total=len(sents), desc="Processing "):
    preds.append(processor.find_word(sent, phrase))

Processing : 100%|██████████| 18/18 [00:58<00:00,  3.26s/it]


In [42]:
acc = 0
for pred, target in zip(preds, targets):
    if not pred and not isinstance(target, str):
        acc += 1
    elif pred and pred[1] == [target]:
        acc += 1
acc / len(targets)

0.8888888888888888

In [44]:
sents = test_df["sentence"].to_list()
phrases = test_df["phrase"].to_list()
targets = test_df["target"].to_list()

In [56]:
preds = []
for sent, phrase in tqdm(zip(sents, phrases), total=len(sents), desc="Processing "):
    preds.append(processor.find_word(sent, phrase))

Processing : 100%|██████████| 421/421 [22:27<00:00,  3.20s/it]


In [80]:
non_null_targets = [targ for targ in targets if isinstance(targ, str)]

In [81]:
len(non_null_targets)

856

In [82]:
acc = 0
acc_words = 0
for pred, target in zip(preds, targets):
    if not pred and not isinstance(target, str):
        acc += 1
    elif pred and isinstance(target, str) and target.lower() in pred[1][0].lower().strip("."):
        acc_words += 1
        acc += 1
acc / len(targets), acc_words / len(non_null_targets)

(0.746, 0.7289719626168224)

In [79]:
acc_sent = 0
for pred, target in zip(preds, targets):
    if not pred and not isinstance(target, str):
        acc_sent += 1
    elif pred and isinstance(target, str):
        acc_sent += 1

acc_sent / len(targets)

0.879